# COVID-19 data reader

Questo notebook carica i dati regionali riguardanti la diffusione e la gestione del COVID-19 direttamente dal repository delle Protezione civile.

I dati sono così organizzati:
- per ciascuna feature si crea un dizionario i cui elementi sono le regioni, accessibili per nome (es. `trend[indicatore][regione]`)
- ogni elemento contiene il valore misurato nei giorni di osservazione
- le date sono nella lista di stringhe `giorni`
- è stata creata una entry per ciascun dizionario dell'indicatore per l'*Italia*. Questo indicatore, per quanto riguarda gli indicatori di base originali, ne è la somma, sltrimenti è opportunamente calcolato.


Fonte dati: *Presidenza del Consiglio dei Ministri - Dipartimento della Protezione Civile*

https://github.com/pcm-dpc/COVID-19

> **NOTA** Questo notebook nasce per condividere il proprio kernel con altri notebooks dove vengono effettuate e visualizzate le analisi vere e proprie.

## Lista indicatori raccolti nella struttura dati

**Originali:**
- ricoverati_con_sintomi
- terapia_intensiva
- totale_ospedalizzati
- isolamento_domiciliare
- totale_positivi
- variazione_totale_positivi
- nuovi_positivi
- dimessi_guariti
- deceduti
- totale_casi
- tamponi
- casi_da_sospetto_diagnostico
- casi_da_screening

**Derivati:**
- tamponi_giornalieri
- guariti_giornalieri
- casi_da_sospetto_diagnostico_giornalieri
- casi_da_screening_giornalieri
- deceduti_giornalieri
- perc_tamponi_positivi
- perc_guariti_su_positivi
- perc_deceduti_su_positivi

**Altre variabili:**
- giorni
- day_counter
- regioni
- eventi (dizionario di eventi notevoli con data)
    - data (str)
    - id_giorno (indice del giorno dell'evento)
    - desc (str)
- posti in TI per regione

In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle as pkl


import warnings
warnings.filterwarnings('ignore')

## Parametri

In [2]:
#data_file='dpc-covid19-ita-regioni.csv' # locale non aggiornato
data_file='https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

## Lettura dati e preprocessing

In [3]:
data=pd.read_csv(data_file)
print(data.columns)

giorni=pd.unique(data['data'])
day_counter=list(range(len(giorni)))
print('Giorni osservati: %d'%len(giorni))
print('Da: %s'%giorni[0][0:10])
print(' A: %s'%giorni[-1][0:10])
regioni=pd.unique(data['denominazione_regione'])

#data[data['denominazione_regione']=='Lombardia'].head(5)
data[data['denominazione_regione']=='Toscana'].tail(5)


Index(['data', 'stato', 'codice_regione', 'denominazione_regione', 'lat',
       'long', 'ricoverati_con_sintomi', 'terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'casi_da_sospetto_diagnostico', 'casi_da_screening',
       'totale_casi', 'tamponi', 'casi_testati', 'note',
       'ingressi_terapia_intensiva', 'note_test', 'note_casi',
       'totale_positivi_test_molecolare',
       'totale_positivi_test_antigenico_rapido', 'tamponi_test_molecolare',
       'tamponi_test_antigenico_rapido', 'codice_nuts_1', 'codice_nuts_2'],
      dtype='object')
Giorni osservati: 952
Da: 2020-02-24
 A: 2022-10-02


,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
19904,2022-09-28T17:00:00,ITA,9,Toscana,43.769231,11.255889,191,2,193,84114,...,NaN,1.0,NaN,NaN,680941.0,721266.0,6957437.0,8560421.0,ITI,ITI1
19925,2022-09-29T17:00:00,ITA,9,Toscana,43.769231,11.255889,208,4,212,84743,...,NaN,3.0,NaN,NaN,681278.0,722994.0,6958473.0,8570412.0,ITI,ITI1
19946,2022-09-30T17:00:00,ITA,9,Toscana,43.769231,11.255889,227,4,231,35230,...,"A seguito di adeguamento informatico, con il m...",0.0,NaN,NaN,681593.0,724500.0,6959356.0,8578828.0,ITI,ITI1
19967,2022-10-01T17:00:00,ITA,9,Toscana,43.769231,11.255889,233,4,237,36287,...,NaN,0.0,NaN,NaN,681925.0,726051.0,6960535.0,8587042.0,ITI,ITI1
19988,2022-10-02T17:00:00,ITA,9,Toscana,43.769231,11.255889,238,5,243,37051,...,NaN,3.0,NaN,NaN,682218.0,727272.0,6961524.0,8594890.0,ITI,ITI1


### Lettura dati popolazione

La popolazione è ri-calcolata di tanto in tanto da uno scripr esterno che memorizza il risultato in un file pickle.

In [4]:
pop_file=open('popolazione.pkl','rb')
popolazione=pkl.load(pop_file)
pop_file.close()

for r in popolazione.keys():
    print('%15s: %d'%(r,popolazione[r]))

        Abruzzo: 1293941
     Basilicata: 553254
       Calabria: 1894110
       Campania: 5712143
 Emilia-Romagna: 4464119
Friuli Venezia Giulia: 1206216
          Lazio: 5755700
        Liguria: 1524826
      Lombardia: 10027602
         Marche: 1512672
         Molise: 300516
       Piemonte: 4311217
         Puglia: 3953305
       Sardegna: 1611621
        Sicilia: 4875290
        Toscana: 3692555
         Umbria: 870165
  Valle d'Aosta: 125034
         Veneto: 4879133
    P.A. Trento: 545425
   P.A. Bolzano: 532644


### Inserimento dati capacità TI

In [5]:
posti_ti_covid={'Lombardia': 1446,
 'Lazio': 845,
 'Campania': 834,
 'Sicilia': 719,
 'Veneto': 705,
 'Emilia-Romagna': 646,
 'Piemonte': 626,
 'Puglia': 580,
 'Toscana': 567,
 'Calabria': 280,
 'Sardegna': 235,
 'Liguria': 237,
 'Marche': 220,
 'Abruzzo': 189,
 'Friuli Venezia Giulia': 175,
 'P.A. Trento': 78,
 'Umbria': 128,
 'Basilicata': 81,
 'Molise': 44,
 "Valle d'Aosta": 20,
 'P.A. Bolzano': 77,
 'Italia':8732}

posti_ti_antecovid={'Lombardia': 861,
 'Lazio': 571,
 'Campania': 335,
 'Sicilia': 418,
 'Veneto': 494,
 'Emilia-Romagna': 449,
 'Piemonte': 327,
 'Puglia': 304,
 'Toscana': 374,
 'Calabria': 146,
 'Sardegna': 134,
 'Liguria': 180,
 'Marche': 115,
 'Abruzzo': 123,
 'Friuli Venezia Giulia': 120,
 'P.A. Trento': 32,
 'Umbria': 70,
 'Basilicata': 49,
 'Molise': 30,
 "Valle d'Aosta": 10,
 'P.A. Bolzano': 37,
 'Italia':5179}



# si stima che GLOBALMENTE siano stati approntati il 38% dei nuovi posti TI
posti_ti_stimati={}
for r in regioni:
    posti_ti_stimati[r]=np.round(posti_ti_antecovid[r]+(posti_ti_covid[r]-posti_ti_antecovid[r])*.38)
    
posti_ti_stimati['Italia']=np.round(posti_ti_antecovid['Italia']+(posti_ti_covid['Italia']-posti_ti_antecovid['Italia'])*.38)

### Inserimento di eventi notevoli riferiti a date

In [6]:
lockdown={'data':'2020-03-10',
          'id_giorno':15,
          'desc':'Lockdown'}

scuole_chiuse={'data':'2020-03-04',
               'id_giorno':9,
               'desc':'Scuole chiuse'}

fase_2={'data':'2020-05-04',
               'id_giorno':71,
               'desc':'Fase2'}


eventi=[scuole_chiuse,lockdown, fase_2]

## Analisi

### Creazione della struttura dati

- Si crea un dizionario per ciascuno degli indicatori selezionati
- a sua volta ciascun elemento del primo dizionario è un dizionario con le regioni come chiave
    - ogni elemento indicatore contiene i valori dell'indicatore per la regione

**Esempio:**
```    
trend[indicatore][regione]
```

In [7]:
# indicatori osservati
indicatori=['ricoverati_con_sintomi', 'terapia_intensiva', 'ingressi_terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'totale_casi', 'tamponi',
        'casi_da_sospetto_diagnostico', 'casi_da_screening']

# qui immetto tutti i trend per ciascuna regione
trend={}
for ind in indicatori:
    trend[ind]={}
    for regione in regioni:
        trend[ind][regione]=np.array(data[data['denominazione_regione']==regione][ind])

# calcolo gli indicatori nazionali come somma delle regioni
for ind in indicatori:
    trend[ind]['Italia']=np.zeros(len(giorni))
    for regione in regioni:
        trend[ind]['Italia']+=trend[ind][regione]

### Indicatori derivati

#### Giorno della settimana associato ai giorni

In [8]:
def which_weekday(datestring):
    # datestring is in this format '2020-02-24T18:00:00'
    return datetime.date(int(datestring[0:4]),int(datestring[5:7]),int(datestring[8:10])).weekday()

giorno_settimana=[which_weekday(d) for d in giorni]

#### Tamponi giornalieri

In [9]:
# tamponi giornalieri
trend['tamponi_giornalieri']={}
for regione in regioni:
    trend['tamponi_giornalieri'][regione]=np.insert(np.diff(trend['tamponi'][regione]),0,0)

trend['tamponi_giornalieri']['Italia']=np.insert(np.diff(trend['tamponi']['Italia']),0,0)


#### Provenienza infetti giornaliera
> screening o sintomi

In [10]:
trend['casi_da_sospetto_diagnostico_giornalieri']={}
trend['casi_da_screening_giornalieri']={}

for regione in regioni:
    trend['casi_da_sospetto_diagnostico_giornalieri'][regione]=np.insert(np.diff(trend['casi_da_sospetto_diagnostico'][regione]),0,0)
    trend['casi_da_screening_giornalieri'][regione]=np.insert(np.diff(trend['casi_da_screening'][regione]),0,0)

trend['casi_da_sospetto_diagnostico_giornalieri']['Italia']=np.insert(np.diff(trend['casi_da_sospetto_diagnostico']['Italia']),0,0)
trend['casi_da_screening_giornalieri']['Italia']=np.insert(np.diff(trend['casi_da_screening']['Italia']),0,0)


#### Deceduti giornalieri

In [11]:
# deceduti giornalieri
trend['deceduti_giornalieri']={}
for regione in regioni:
    trend['deceduti_giornalieri'][regione]=np.insert(np.diff(trend['deceduti'][regione]),0,0)

trend['deceduti_giornalieri']['Italia']=np.insert(np.diff(trend['deceduti']['Italia']),0,0)


#### Guariti giornalieri

In [12]:
trend['guariti_giornalieri']={}
for regione in regioni:
    trend['guariti_giornalieri'][regione]=np.insert(np.diff(trend['dimessi_guariti'][regione]),0,0)

trend['guariti_giornalieri']['Italia']=np.insert(np.diff(trend['dimessi_guariti']['Italia']),0,0)


#### Percentuale tamponi positivi

In [13]:
# percentuale tamponi positivi su quelli effettuati
trend['perc_tamponi_positivi']={}
for regione in regioni:
    trend['perc_tamponi_positivi'][regione]=100*trend['nuovi_positivi'][regione]/trend['tamponi_giornalieri'][regione]

trend['perc_tamponi_positivi']['Italia']=100*trend['nuovi_positivi']['Italia']/trend['tamponi_giornalieri']['Italia']

#### Rate deceduti e rate guariti su attualmente positivi (il giorno precedente)

In [14]:
trend['perc_guariti_su_positivi']={}
trend['perc_deceduti_su_positivi']={}

np.hstack((regioni,'Italia'))

for regione in np.hstack((regioni,'Italia')):
    # percentuale di GUARITI su POSITIVI
    trend['perc_guariti_su_positivi'][regione]=np.zeros_like(trend['totale_positivi'][regione],dtype=float)
    for d in range(1,len(giorni)):
        try:
            trend['perc_guariti_su_positivi'][regione][d]=100*float(trend['guariti_giornalieri'][regione][d]/trend['totale_positivi'][regione][d-1])
            #print('%d   %4.2f  %4.2f'%(d,trend['guariti_giornalieri'][regione][d],trend['totale_positivi'][regione][d-1]))
        except ValueError:
            trend['perc_guariti_su_positivi'][regione][d]=np.NaN
            
    # percentuale di DECEDUTI su POSITIVI
    trend['perc_deceduti_su_positivi'][regione]=np.zeros_like(trend['totale_positivi'][regione],dtype=float)
    for d in range(1,len(giorni)):
        try:
            trend['perc_deceduti_su_positivi'][regione][d]=100*float(trend['deceduti_giornalieri'][regione][d]/trend['totale_positivi'][regione][d-1])
            #print('%d   %4.2f  %4.2f'%(d,trend['deceduti_giornalieri'][regione][d],trend['totale_positivi'][regione][d-1]))
        except ValueError:
            trend['perc_deceduti_su_positivi'][regione][d]=np.NaN
                

## Salvataggio dati su file PICKLE

Vengono salvati su file **pickle** nell' ordine:
- trend
- regioni
- giorni


In [15]:
output_data_file='stored_data.pkl'

outfile = open(output_data_file,'wb')
pkl.dump(trend,outfile)
pkl.dump(regioni,outfile)
pkl.dump(giorni,outfile)
pkl.dump(giorno_settimana,outfile)
pkl.dump(popolazione,outfile)
pkl.dump(posti_ti_covid,outfile)
pkl.dump(posti_ti_antecovid,outfile)
pkl.dump(posti_ti_stimati,outfile)
outfile.close()